In [36]:
psql_params = {
    'address': 'host',
    'db_name': 'dbname',
    'port': '5432',
}

psql_auth = {
    'auth_user': 'username',
    'auth_pw': 'userpassword'
}

In [35]:
import psycopg2
from psycopg2.extras import execute_batch


class psqlConnector:
    def __init__(self):
        self.conn = psycopg2.connect(
            host = psql_params['address'],
            dbname = psql_params['db_name'],
            user = psql_auth['auth_user'],
            password = psql_auth['auth_pw'],
            port = psql_params['port']
        )

        
    def _database(self):
        self.cur = self.conn.cursor()
        return self.cur

    
    def select_data(self):
        pass

    
    def insert_data(self, query, values):
        _cur = self._database()
        try:
            _cur.execute(query, values)
            self.conn.commit()

            print(f">>> Successfully inserted data into table")

        except psycopg2.Error as e:
            self.conn.rollback()
            print(f">>> failed insert data into table: {e}")
        
        else:
            _cur.close()
        
        
    def insert_bulk_data(self, query, values): # 해당 함수는 좀 더 보완이 필요함 
        _cur = self._database()
        try:
            execute_batch(_cur, query, values)
            self.conn.commit()
            _cur.close()
            print(">>> Successfully inserted data into table")

        except psycopg2.Error as e:
            self.conn.rollback()
            print(f">>> failed insert data into table: {e}")


    def update_date(self, data, attribute):
        pass
    
    
    def disconnect(self):
        if self.conn:
            self.conn.close()
            print(">>> db disconnected!!")

In [15]:
import requests
import datetime

In [16]:
from yarl import URL

In [17]:
HEADERS = {
    'Accept': 'application/vnd.github+json',
    'Authorization': 'Bearer <github access token>',
    'X-GitHub-Api-Version': '2022-11-28'
}

In [18]:
def get_json(url):
    """
    url로 api 요청을 보내고, 응답값을 json 형태로 반환하는 함수입니다.
    
    Args:
        url (str): 요청을 보낼 url 주소
    
    Returns:
        dict or list(dict)
    """
    req = requests.get(url, headers=HEADERS)
    return req.json()

In [19]:
def get_current_time():
    """
    현재 시간과 날짜를 문자열 형태로 반환하는 함수입니다.
    
    Returns:
        str -> 2023-05-30 18:42:24
    """
    current_datetime = datetime.datetime.now()
    return current_datetime.strftime("%Y-%m-%d %H:%M:%S")

In [20]:
def clean_orgs_data(repo_content):
    """
    organization api 요청 응답 값을 DB에 적재하기 알맞은 형태로 정제하는 함수입니다.
    
    Returns:
        dict
    """
    return {
    'orgs_id': repo_content['id'],
    'node_id': repo_content['node_id'],
    'name': repo_content['name'],
    'description': repo_content['description'],
    'company': repo_content['company'],
    'blog': repo_content['blog'],
    'location': repo_content['location'],
    'email': repo_content['email'],
    'twitter_username': repo_content['twitter_username'],
    'followers': repo_content['followers'],
    'following': repo_content['following'],
    'is_verified': repo_content['is_verified'],
    'has_organization_projects': repo_content['has_organization_projects'],
    'has_repository_projects': repo_content['has_repository_projects'],
    'public_repos': repo_content['public_repos'],
    'public_gists': repo_content['public_gists'],
    'html_url': repo_content['html_url'],
    'avatar_url': repo_content['avatar_url'],
    'type': repo_content['type'],
    'created_at': repo_content['created_at'],
    'updated_at': repo_content['updated_at'],
    'called_at': get_current_time()
}

#### SQL 문

In [21]:
API_ORGS_TABLE_INSERT_SQL = "INSERT INTO adhoc.api_orgs (orgs_id, node_id, name,\
                  description, company, blog, location, email, twitter_username,\
                  followers, following, is_verified, has_organization_projects, \
                  has_repository_projects, public_repos, public_gists, html_url, \
                  avatar_url, type, created_at, updated_at, called_at) \
                      VALUES (%(orgs_id)s, %(node_id)s, %(name)s, %(description)s, \
                      %(company)s, %(blog)s, %(location)s, %(email)s, %(twitter_username)s, \
                      %(followers)s, %(following)s, %(is_verified)s, %(has_organization_projects)s,\
                      %(has_repository_projects)s, %(public_repos)s, %(public_gists)s, %(html_url)s,\
                      %(avatar_url)s, %(type)s, %(created_at)s, %(updated_at)s, %(called_at)s);"

In [22]:
# users 데이터는 형태 없으므로 위 api_orgs_table_insert_sql 값만 사용합니다.
API_USERS_TABLE_INSERT_SQL = "INSERT INTO adhoc.api_users (users_id, node_id, name,\
                  company, blog, location, email, bio, twitter_username,\
                  followers, following, public_repos, public_gists, html_url, \
                  avatar_url, type, created_at, updated_at, called_at) \
                      VALUES (%(users_id)s, %(node_id)s, %(name)s, %(company)s, \
                       %(blog)s, %(location)s, %(email)s, %(bio)s, %(twitter_username)s, \
                      %(followers)s, %(following)s, %(public_repos)s, %(public_gists)s, %(html_url)s,\
                      %(avatar_url)s, %(type)s, %(created_at)s, %(updated_at)s, %(called_at)s);"

### main 함수

In [33]:
def main():
    db = psqlConnector()
    orgs = ['moloco', 'woowabros', 'daangn', 'toss', \
    'ncsoft', 'line', 'kakao', 'naver', 'nhn']
    for repo in orgs:
        repo_url = URL('https://api.github.com').with_path(f'orgs/{repo}')
        json_data = get_json(repo_url)
        values = clean_orgs_data(json_data)
        if values['name'] is None: # name이 없을 경우 명시적으로 회사명 입력하기
            values['name'] = repo
        db.insert_data(API_ORGS_TABLE_INSERT_SQL, values)
    db.disconnect()

In [34]:
main()

>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> Successfully inserted data into table
>>> db disconnected!!
